In [2]:
import argparse
import sys
import os
import json
import pprint
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import re
import torch
from collections import Counter
from transformers import AutoModel, AutoTokenizer
from tqdm import tqdm
from sklearn.metrics import f1_score

/Users/katemarg/miniconda3/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
BASE_DIR = "/Users/katemarg/PycharmProjects/temporal_robustness_evaluation"
DATA_DIR = os.path.join(BASE_DIR, 'data')
CKPT_DIR = os.path.join(BASE_DIR, 'pretrained_models')
RES_DIR = os.path.join(BASE_DIR, 'results')
LOG_DIR = os.path.join(BASE_DIR, 'logs')
CACHE_DIR = os.path.join(BASE_DIR, 'cached')

TEMPLAMA_ORIG_DIR = os.path.join("data", "templama", "test.json")
TEMPLAMA_REPRODUCED_DIR = os.path.join("templama_docker", "reproduce1", "templama", "test.jsonl")
TEMPLAMA_NEW_DIR = os.path.join(DATA_DIR, "dynamic-templama",
                                "dataset_from_2019-1-1_to_2022-12-31_per_quarter", 
                                "test.jsonl")

dataset_filepath=CACHE_DIR+'/cardiffnlp-twitter-roberta-base-2021-124m_dynamic-templama_multiple_masks.pt'
data_dict_multi_token = torch.load(dataset_filepath)


lm = "cardiffnlp/twitter-roberta-base-mar2022"
tokenizer = AutoTokenizer.from_pretrained(lm, use_fast=False, add_prefix_space=True)
mask_token = "<mask>"

## This notebook is used for error analysis.

In [4]:
# Load pre-trained model tokenizer (vocabulary)
tokenizer = AutoTokenizer.from_pretrained(lm, use_fast=False, add_prefix_space=True)

CLS = tokenizer.cls_token
PAD = tokenizer.pad_token
SEP = tokenizer.sep_token
MASK = tokenizer.mask_token

mask_id = tokenizer.mask_token_id
sep_id = tokenizer.sep_token_id
cls_id = tokenizer.cls_token_id
pad_id = tokenizer.pad_token_id

special_ids = [mask_id, sep_id, cls_id, pad_id]
def tokenizer_return_id(text, filter_special_tokens=False):
    """
    Text to token ids for a string.
    """
    output = tokenizer(text)
    if filter_special_tokens:
        token_ids = [i for i in output['input_ids'] if i not in tokenizer.all_special_ids]
    else:
        token_ids = [i for i in output['input_ids'] ]
    return token_ids

def tokenize_batch(batch):
    """
    Text to token ids for a list of strings.
    """
#     return [tokenizer.convert_tokens_to_ids(sent) for sent in batch]
    return [tokenizer_return_id(sent) for sent in batch]

def untokenize_id(ids):
    """
    Token ids to strings.
    """
#     return [tokenizer.convert_tokens_to_ids(sent) for sent in batch]
    return [tokenizer.decode(_id) for _id in ids]


def untokenize_batch(batch, filter_special_tokens=False):
    """
    Token ids to strings for a list of ids.
    """
#     return [tokenizer.convert_ids_to_tokens(sent) for sent in batch]
#     print(label_id)
    if filter_special_tokens:
        _batch = []
        for sent in batch:
            _batch.append([x for x in sent if x not in special_ids])
        batch = _batch
#         return [tokenizer.decode(label_id) for label_id in batch if label_id not in special_ids]
#     else:
    return [untokenize_id(sent) for sent in batch]

def detokenize(sent):
    """ Roughly detokenizes (mainly undoes wordpiece) """
    # not sure what this does.... their code
    new_sent = []
    for i, tok in enumerate(sent):
        if tok.startswith("##"):
            new_sent[len(new_sent) - 1] = new_sent[len(new_sent) - 1] + tok[2:]
        else:
            new_sent.append(tok)
    return new_sent

In [5]:
def split_dataset(data):
    """
    Split temporal dataset Dt to D_unchanged, D_new and D_updated compared to D_(t-1) for all t.
    Specifically:
    - D_unchanged: data where text_t = text_(t-1) & label_t = label_(t-1)
    - D_updated: data where text_t = text_(t-1) & label_t != label_(t-1)
    - D_new: data where text_t not in D_(t-1)
    - D_deleted: data that exist in D_(t-1) but not in D_t

    Args:
        data: a dictionary with keys the time (year/quarter/month) and values dictionaries
        data = {
                '2019-Q1':
                    {
                    'text': [list of text],
                    'labels': [list of labels],
                    'labels_ids': [list of label token ids -- for a given model/tokenizer],
                    'relations' [list of Wikidata relations]
                    },
                '2019-Q2': {...}
                }

    Returns:
        D_unchanged, D_new, D_updated, D_deleted
    """
    unchanged_t, new_t, updated_t, deleted_t = {}, {}, {}, {}
    quarters = list(data.keys())
    t_0 = quarters[0]  # t=t0
    t_1 = quarters[0]  # t-1

    for t in quarters[1:]:
        if t in ['2022-Q3', '2022-Q4']:
            continue # skip last two quarters of 2022
        data_t = data[t]      # D_t
        data_t_1 = data[t_1]  # D_(t-1)

        unchanged_t[t] = {key: [] for key in data_t.keys()}
        new_t[t] = {key: [] for key in data_t.keys()}
        updated_t[t] = {key: [] for key in data_t.keys()}
        deleted_t[t] = {key: [] for key in data_t.keys()}

        for i in range(0, len(data_t['text'])):
            text_t = data_t['text'][i]
            labels_ids_t = data_t['labels_ids'][i][0]
            if text_t in data_t_1['text']:
                t_1_index = data_t_1['text'].index(text_t)
                labels_inds_t_1 = data_t_1['labels_ids'][t_1_index][0]
                if labels_ids_t == labels_inds_t_1:
                    # text_t = text_t-1 & label_t = label_t-1
                    # add to D_unchanged
                    for key in data_t.keys():
                        unchanged_t[t][key].append(data_t[key][i])
                else:
                    # text_t = text_(t-1) & label_t != label_(t-1)
                    # add to D_updated
                    for key in ['text', 'relation']:
                        updated_t[t][key].append(data_t[key][i])
            else:
                # text_t not in D_(t-1) texts
                # add to D_new
                for key in data_t.keys():
                    new_t[t][key].append(data_t[key][i])
        for j in range(0, len(data_t_1['text'])):
            text_t_1 = data_t_1['text'][j]
            if text_t_1 not in data_t['text']:
                # text_(t+1) not in D_t
                # add to D_deleted
                for key in data_t_1.keys():
                    deleted_t[t][key].append(data_t_1[key][j])
        t_1 = t

        assert len(data_t['text']) == len(unchanged_t[t]['text']) + len(updated_t[t]['text']) + len(new_t[t]['text'])
        print(
            't={}: From total {} samples in D_t, {} are unchanged, {} are updated, {} are deleted and {} are new, compared to D_(t-1).'.format(
                t,
                len(data_t['text']),
                len(unchanged_t[t]['text']),
                len(updated_t[t]['text']),
                len(deleted_t[t]['text']),
                len(new_t[t]['text'])))
    return unchanged_t, new_t, updated_t, deleted_t, data[t_0]

In [6]:
# Split dataset
unchanged_t, new_t, updated_t, deleted_t, orig = split_dataset(data_dict_multi_token)

t=2019-Q2: From total 11237 samples in D_t, 10882 are unchanged, 55 are updated, 301 are deleted and 300 are new, compared to D_(t-1).
t=2019-Q3: From total 11279 samples in D_t, 10710 are unchanged, 84 are updated, 442 are deleted and 485 are new, compared to D_(t-1).
t=2019-Q4: From total 11411 samples in D_t, 10866 are unchanged, 69 are updated, 344 are deleted and 476 are new, compared to D_(t-1).
t=2020-Q1: From total 11322 samples in D_t, 10512 are unchanged, 129 are updated, 771 are deleted and 681 are new, compared to D_(t-1).
t=2020-Q2: From total 11304 samples in D_t, 11060 are unchanged, 39 are updated, 223 are deleted and 205 are new, compared to D_(t-1).
t=2020-Q3: From total 11345 samples in D_t, 10991 are unchanged, 52 are updated, 261 are deleted and 302 are new, compared to D_(t-1).
t=2020-Q4: From total 11325 samples in D_t, 11001 are unchanged, 37 are updated, 307 are deleted and 287 are new, compared to D_(t-1).
t=2021-Q1: From total 11237 samples in D_t, 10582 are 

In [7]:
unchanged_t

{'2019-Q2': {'text': [['Alex Morgan plays for <mask> <mask> <mask> <mask> <mask> <mask> <mask>.'],
   ['Lionel Messi plays for <mask> <mask>.'],
   ['Cristiano Ronaldo plays for <mask> <mask> <mask> <mask> <mask>.'],
   ['LeBron James plays for <mask> <mask> <mask>.'],
   ['Puck Moonen plays for <mask> <mask> <mask> <mask> <mask>.'],
   ['Zlatan Ibrahimović plays for <mask> <mask> <mask> <mask> <mask>.'],
   ['Neymar plays for <mask> <mask> <mask> <mask>.'],
   ['Ronaldinho plays for <mask> <mask> <mask> <mask> <mask> <mask>.'],
   ["Samuel Eto'o plays for <mask> <mask>."],
   ['Megan Rapinoe plays for <mask> <mask>.'],
   ['Alisha Lehmann plays for <mask> <mask> <mask> <mask> <mask> <mask> <mask>.'],
   ['Robert Lewandowski plays for <mask> <mask> <mask>.'],
   ['Gianluigi Buffon plays for <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask> <mask>.'],
   ['Andrea Pirlo plays for <mask> <mask> <mask> <mask>.'],
   ['Kevin Durant plays for <mask> <mask>.'],
   ['Lukas Podolsk

## Multi-token generation

In [8]:
from transformers import AutoTokenizer, AutoModel, pipeline
lm = "cardiffnlp/twitter-roberta-base-mar2022"
fill_mask_model = pipeline(
    'fill-mask', model=lm, framework="pt",
    tokenizer=tokenizer, top_k=100
)
model = fill_mask_model.model
model.eval()
cuda = torch.cuda.is_available()
if cuda:
    model = model.cuda()
year='2019-Q1'
batch_size = 100
N = batch_size # number of shots
temperature = 1.0
burnin = 200

In [485]:
def generate_step(out, gen_idx, temperature=None, top_k=0, sample=False, return_list=True, num_masks=0):
    """ Generate a word from out[gen_idx]

    args:
        - out (torch.Tensor): tensor of logits of size batch_size x seq_len x vocab_size
        - gen_idx (int): location for which to generate for
        - top_k (int): if >0, only sample from the top k most probable words
        - sample (Bool): if True, sample from full distribution. Overridden by top_k
    """
    logits = out[:, gen_idx]
    if num_masks==1:  # if single-token then return the argmax
        idx = torch.argmax(logits, dim=-1)
        return idx.tolist() if return_list else idx
    else:
        if temperature is not None:
            logits = logits / temperature
        if top_k > 0:
            kth_vals, kth_idx = logits.topk(top_k, dim=-1)
            dist = torch.distributions.categorical.Categorical(logits=kth_vals)
            idx = kth_idx.gather(dim=1, index=dist.sample().unsqueeze(-1)).squeeze(-1)
        elif sample:
            dist = torch.distributions.categorical.Categorical(logits=logits)
            idx = dist.sample().squeeze(-1)
        else:
            idx = torch.argmax(logits, dim=-1)
        return idx.tolist() if return_list else idx

In [486]:
text_list = data_dict_multi_token[year]['text']
labels_list = data_dict_multi_token[year]['labels']
labels_ids_list = data_dict_multi_token[year]['labels_ids']
relation_list = data_dict_multi_token[year]['relation']
num_answers_list = data_dict_multi_token[year]['num_answers']

### For a single test example:

In [488]:
i = 0  # for i in range(0, len(text_list)) i is in [0,#test examples]

In [489]:
len(text_list)

11238

In [497]:
text_i = text_list[i]
# text_i = ['The name of the current president of the United States is <mask> <mask>.']
labels_i = labels_list[i][0]
labels_ids_i = labels_ids_list[i][0] # there is an 'extra' list that is why we put [0]
relation_i = relation_list[i]
num_answers_i = num_answers_list[i]
print('Example: {}, Labels: {}, Ids: {}'.format(text_i, labels_i, labels_ids_i))

Example: ['Alex Morgan plays for <mask> <mask> <mask> <mask> <mask> <mask> <mask>.'], Labels: [" United States women's national soccer team"], Ids: [315, 532, 390, 18, 632, 4191, 165]


## We have the test example: "Alex Morgan plays for X." where the object (team) is " United States women's national soccer team". 

We tokenize the gold label string and this results in 7 masks, with the following token ids `[315, 532, 390, 18, 632, 4191, 165]`. 

We thus prompt the model with `"Alex Morgan plays for <mask> <mask> <mask> <mask> <mask> <mask> <mask>."` and find the most probable `N=100` generated token sequences of length M=7.

In [498]:
generated_seq_tokens = [[] for _ in range(batch_size)]  # for each trial
logits_list = [[] for _ in range(batch_size)]

In [499]:
# tokenize text
tokenized_sentence = tokenizer_return_id(text_i)[0]

# mask indices to generate
mask_inds = list(np.where(np.array(tokenized_sentence) == mask_id)[0])

# number of masks (tokens)
M = len(mask_inds)

# create batch (same input, N times)
tokenized_sentence_b = [tokenized_sentence for _ in range(batch_size)]

In [500]:
mask_inds

[5, 6, 7, 8, 9, 10, 11]

In [501]:
for m in mask_inds:
    # tensor
    inp = torch.tensor(tokenized_sentence_b).cuda() if cuda else torch.tensor(tokenized_sentence_b)

    # get logits
    out = model(inp)
    logits = out.logits  # batch_size x max_len x vocab

    # get new ids
    idxs_b = generate_step(logits, gen_idx=m, top_k=10, temperature=temperature, # sample=(m < burnin)
                          )

    # replace mask with predicted token id
    tokenized_sentence_b_np = np.array(tokenized_sentence_b)                       
    tokenized_sentence_b_np[:,m] = np.array(idxs_b)
    tokenized_sentence_b = tokenized_sentence_b_np.tolist() 

    # the following code does not work *and I don't know why*
    # for jj in range(len(idxs_b)):
    #     print('before: {}, predicted token id: {}'.format(tokenized_sentence_b[jj][m],idxs_b[jj]))
    #     tokenized_sentence_b[jj][m] = idxs_b[jj]

    assert sorted(idxs_b) == sorted([sent[m] for sent in tokenized_sentence_b])

    # find logits
    logits_m = logits[:,m]  # logits for the mask in position m
    logits_b = logits_m[:,idxs_b].tolist()[0]  # logits for sampled tokens

    assert len(idxs_b) == len(logits_b)

    # add generated tokens and corresponding logits to lists
    for j in range(N):
        generated_seq_tokens[j].append(idxs_b[j])
        logits_list[j].append(logits_b[j])

In [502]:
# calculate sum of logits for each generated sequence of tokens
sum_logits = np.array(logits_list).sum(axis=-1)

# finding ranking (return indices of the parallel lists for logits and generated tokens)
ranked_inds_of_list = sum_logits.argsort()[::-1]

# ranked logits sum in descending order
ranked_logits = sum_logits[ranked_inds_of_list]

# ranked generated tokens in descending order
ranked_generated_tokens = np.array(generated_seq_tokens)[ranked_inds_of_list]

In [503]:
for no, p in enumerate(ranked_generated_tokens):
    print('{}: {}'.format(no,"".join(untokenize_id(p))))

0:  the @OfficialFAFA right now
1:  the @OfficialBRFC as well
2:  the #AFCAFLs
3:  the U.S. national team
4:  the @CelticFC side
5:  the @LFC and are amazing
6:  the U.K. national team
7:  the #Eagles in this game
8:  the @user @user @user
9:  the @Saints at the moment
10:  the #Rams for the season
11:  the @LFC against #Arsenal
12:  the @Rovers at 10 tonight
13:  the #Eagles at the minute
14:  the @Eagles in a minute
15:  the #Eagles in the game
16:  the @user as the youth manager
17:  the #Bears on the night
18:  the @user on Friday at home
19:  the U.S. Soccer Team
20:  the #Rays in this clip
21:  the @user to start the game
22:  the US national team in this match
23:  the @user today at the Stadium
24:  the @user as his first international
25:  the @Lions tonight in Philly
26:  @RangersFC as the forward
27:  the #LeedsCity side today
28:  the @user in the home opener
29:  @user and @user this weekend
30:  the Red Devils. That's it
31:  @user and @user on Thursday
32:  the #Bills, p

## We can see that the model's 4th most probable prediction is `" the U.S. national team"` which we would expect to be a correct answer. 

However, the actual token ids of this generated sequence are `[5, 121, 4, 104, 4, 632, 165]`, while the 'gold' tokens are `[315, 532, 390, 18, 632, 4191, 165]`

## Evaluation

TempLAMA: "F1 score, which captures the precision 
    and recall that words chosen as being part of the answer are actually part of the answer."

In [504]:
# compare with gold labels
import evaluate
assert len(labels_ids_i) == M
f1_micro_list, f1_macro_list = [], []
rouge_list, bleu_list = [], []

rouge = evaluate.load('rouge')
bleu = evaluate.load("bleu")

In [505]:
labels_ids_i

[315, 532, 390, 18, 632, 4191, 165]

In [1]:
ranked_generated_tokens[3]

NameError: name 'ranked_generated_tokens' is not defined

In [509]:
for j in range(0,N):
    gold_ids = labels_ids_i
    pred_ids_j = ranked_generated_tokens[j]
    gold_tok = labels_i  # list of strings
    pred_tok_j = ["".join(untokenize_id(ranked_generated_tokens[j]))] # list of strings
    
    # F1 score
    # F1_micro calculates metrics globally by counting the total true positives, 
    # false negatives and false positives.
    f1_micro_list.append(f1_score(gold_ids,pred_ids_j, average='micro')) 
    # F1_macro calculates metrics for each label, and finds their unweighted mean. 
    # This does not take label imbalance into account.
    f1_macro_list.append(f1_score(gold_ids,pred_ids_j, average='macro'))
    
    # BLEU
    bleu_list.append(bleu.compute(references=gold_tok,
              predictions=pred_tok_j)['bleu'])

    # ROUGE
    rouge_list.append(rouge.compute(references=gold_tok,
                  predictions=pred_tok_j))
    
    # exact match / P@1
max_f1_micro = np.array(f1_micro_list).max()
max_f1_macro = np.array(f1_macro_list).max()
max_bleu = np.array(bleu_list).max()

In [512]:
f1_macro_list

[0.0,
 0.0,
 0.0,
 0.09090909090909091,
 0.0,
 0.0,
 0.09090909090909091,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.08333333333333333,
 0.0,
 0.08333333333333333,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.07692307692307693,
 0.0,
 0.0,
 0.0,
 0.0,
 0.08333333333333333,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0,
 0.0]

In [234]:
["".join(untokenize_id(ranked_generated_tokens[42]))]

[' luxury cars']

In [235]:
bleu.compute(references=labels_i,
              predictions=["".join(untokenize_id(ranked_generated_tokens[42]))])

{'bleu': 0.0,
 'precisions': [0.0, 0.0, 0.0, 0.0],
 'brevity_penalty': 1.0,
 'length_ratio': 1.0,
 'translation_length': 2,
 'reference_length': 2}

In [236]:
rouge_list

[{'rouge1': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)),
  'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)),
  'rougeL': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)),
  'rougeLsum': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0))},
 {'rouge1': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0)),
  'rouge2': AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasur

In [230]:
np.array(f1_macro_list).max()

0.3333333333333333

In [231]:
np.array(bleu_list).max()

0.0

# All together:

In [299]:
for_logging_dict = {
    'model': None,
    'quarter': None,
    'split': None,
    'logits': None,
    'mlm_scores': None,
    'f1_macro': None,
    'f1_micro': None,
    'bleu': None,
    'rouge':None
    
}
rouge = evaluate.load('rouge')
bleu = evaluate.load("bleu")

max_f1_micro_list, max_f1_macro_list = [], []
max_bleu_list, max_rouge_list = [], []

all_logits_list = []
all_pred_tokens_list = []

for i in tqdm(range(0, 1000)):
    text_i = text_list[i]
    labels_i = labels_list[i][0]
    labels_ids_i = labels_ids_list[i][0] # there is an 'extra' list that is why we put [0]
    relation_i = relation_list[i]
    num_answers_i = num_answers_list[i]
#     print('i={}: Example: {}, Labels: {}, Ids: {}'.format(i, text_i, labels_i, labels_ids_i))
    
    generated_seq_tokens = [[] for _ in range(batch_size)]  # for each trial
    logits_list = [[] for _ in range(batch_size)]

    # tokenize text
    tokenized_sentence = tokenizer_return_id(text_i)[0]

    # mask indices to generate
    mask_inds = list(np.where(np.array(tokenized_sentence) == mask_id)[0])

    # number of masks (tokens)
    M = len(mask_inds)

    # create batch (same input, N times)
    tokenized_sentence_b = [tokenized_sentence for _ in range(batch_size)]

    for m in mask_inds:
        # tensor
        inp = torch.tensor(tokenized_sentence_b).cuda() if cuda else torch.tensor(tokenized_sentence_b)

        # get logits
        out = model(inp)
        logits = out.logits  # batch_size x max_len x vocab

        # get new ids
        idxs_b = generate_step(logits, gen_idx=m, top_k=50, temperature=temperature, # sample=(m < burnin)
                              )

        # replace mask with predicted token id
        tokenized_sentence_b_np = np.array(tokenized_sentence_b)                       
        tokenized_sentence_b_np[:,m] = np.array(idxs_b)
        tokenized_sentence_b = tokenized_sentence_b_np.tolist() 

        # the following code does not work *and I don't know why*
        # for jj in range(len(idxs_b)):
        #     print('before: {}, predicted token id: {}'.format(tokenized_sentence_b[jj][m],idxs_b[jj]))
        #     tokenized_sentence_b[jj][m] = idxs_b[jj]

        assert sorted(idxs_b) == sorted([sent[m] for sent in tokenized_sentence_b])

        # find logits
        logits_m = logits[:,m]  # logits for the mask in position m
        logits_b = logits_m[:,idxs_b].tolist()[0]  # logits for sampled tokens

        assert len(idxs_b) == len(logits_b)

        # add generated tokens and corresponding logits to lists
        for j in range(N):
            generated_seq_tokens[j].append(idxs_b[j])
            logits_list[j].append(logits_b[j])
    
    # calculate sum of logits for each generated sequence of tokens
    sum_logits = np.array(logits_list).sum(axis=-1)

    # finding ranking (return indices of the parallel lists for logits and generated tokens)
    ranked_inds_of_list = sum_logits.argsort()[::-1]

    # ranked logits sum in descending order
    ranked_logits = sum_logits[ranked_inds_of_list]

    # ranked generated tokens in descending order
    ranked_generated_tokens = np.array(generated_seq_tokens)[ranked_inds_of_list]
    
    all_logits_list.append(ranked_logits)
    all_pred_tokens_list.append(ranked_generated_tokens)
    
    # Evaluation
#     print('Evaluation!')
    f1_micro_i, f1_macro_i = [], []
    rouge_i, bleu_i = [], []

    gold_ids = labels_ids_i
    gold_tok = labels_i  # list of strings
    
    for j in range(N):    
        pred_ids_j = ranked_generated_tokens[j]
        pred_tok_j = ["".join(untokenize_id(ranked_generated_tokens[j]))] # list of strings

        # F1 score
        # F1_micro calculates metrics globally by counting the total true positives, 
        # false negatives and false positives.
        f1_micro_i.append(f1_score(gold_ids,pred_ids_j, average='micro')) 
        # F1_macro calculates metrics for each label, and finds their unweighted mean. 
        # This does not take label imbalance into account.
        f1_macro_i.append(f1_score(gold_ids,pred_ids_j, average='macro'))

        # BLEU
#         bleu_i.append(bleu.compute(references=gold_tok,
#                   predictions=pred_tok_j)['bleu'])

#         # ROUGE
#         rouge_i.append(rouge.compute(references=gold_tok,
#                       predictions=pred_tok_j))

        # exact match / P@1
        
    max_f1_micro_list.append(np.array(f1_micro_i).max())
    max_f1_macro_list.append(np.array(f1_macro_i).max())
#     max_bleu_list.append(np.array(bleu_i).max())
    print('Avg f1_micro {}, Avg f1_macro {}'.format(np.mean(max_f1_micro_list), np.mean(max_f1_macro_list)))


  0%|                                                                                                      | 1/1000 [00:03<56:53,  3.42s/it]

Avg f1_micro 0.14285714285714285, Avg f_macro 0.07692307692307693


  0%|▏                                                                                                     | 2/1000 [00:04<29:59,  1.80s/it]

Avg f1_micro 0.07142857142857142, Avg f_macro 0.038461538461538464


  0%|▎                                                                                                     | 3/1000 [00:06<33:08,  1.99s/it]

...

Avg f1_micro 0.12951595832357357, Avg f_macro 0.07611831941255644


100%|█████████████████████████████████████████████████████████████████████████████████████████████████▉| 999/1000 [1:56:56<00:03,  3.81s/it]

Avg f1_micro 0.12952931286264618, Avg f_macro 0.07613312498963004


100%|█████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [1:56:58<00:00,  7.02s/it]

Avg f1_micro 0.12959978354978355, Avg f_macro 0.0761819918646404


In [294]:
np.mean([36, 46, 35, 345])

115.5

In [290]:
["".join(untokenize_id(c)) for c in ranked_generated_tokens]

[' the first time in years',
 ' the first time since 2015',
 ' the first time since 2015',
 ' the best player in history',
 ' his national team in Spain',
 ' the football team of America',
 ' the Brazilian team. Legend',
 " Ronaldo's team in 2020",
 ' the Brazilian team on field',
 ' a different team every game',
 ' the national anthem for Brazil',
 ' the World Cup of Football',
 ' the United States of America',
 ' the United States of America',
 ' the United States of America',
 ' the United States of America',
 ' the United States of America',
 ' the National team or not',
 ' Real Madrid and never scores',
 ' the best of his teams',
 ' the Ronaldo.  Nice',
 ' the club, not himself',
 ' Real Madrid. ��',
 ' the champions league right now',
 ' the Madrid Madrid football team',
 ' the Real Madrid Real Madrid',
 ' @user today in Argentina',
 ' the reds, apparently',
 ' Portugal, and not Argentina',
 ' a team in their league',
 ' this world Cup as well',
 ' the best of his ability',
 ' th

In [238]:
rouge.compute(references=gold_tok,
                      predictions=pred_tok_j)['rouge1']

AggregateScore(low=Score(precision=0.0, recall=0.0, fmeasure=0.0), mid=Score(precision=0.0, recall=0.0, fmeasure=0.0), high=Score(precision=0.0, recall=0.0, fmeasure=0.0))

In [32]:
np.array(f1_macro).max()

0.07692307692307693

In [25]:
untokenize_id(labels_ids_i)

[' United', ' States', ' women', "'s", ' national', ' soccer', ' team']

In [33]:
untokenize_batch(ranked_generated_tokens[0:10])

[[' the', ' St', '.', ' Louis', ' Louis', ' Blues', ' tonight'],
 [' the', ' @', 'All', 'S', ' considered', 'FC', ' today'],
 [' the', ' game', '.', ' ', ' That', "'s", ' all'],
 [' the', ' @', 'D', 'ynam', 'ites', ' FC', ' today'],
 [' the', ' Panthers', '!', ' 1', '-', '0', ' Panther'],
 [' the', ' @', 'Chief', 's', ' in', ' the', ' fourth'],
 [' the', ' Bills', ' in', ' my', ' head', ' right', ' now'],
 [' the', ' @', 'Rap', 'She', 'et', ' right', ' now'],
 [' the', ' Bears', '.', ' Will', ' be', ' tough', ' loss'],
 [' the', ' #', 'Fal', 'cons', ' on', ' both', ' sides']]

In [ ]:
ranked_logits = sum_logits[ranked_inds_of_list]

In [ ]:
ranked_generated_tokens = np.array(generated_seq_tokens)[ranked_inds_of_list]

In [ ]:
ranked_generated_tokens

In [ ]:
# generate token for each mask in parallel
for m in mask_inds:
    # tensor
    inp = torch.tensor(tokenized_sentence_b).cuda() if cuda else torch.tensor(tokenized_sentence_b)  # shape bs x max_len
    
    # get logits
    out = model(inp)
    logits = out.logits
    
    # get new ids
    idxs = generate_step(logits, gen_idx=m, 
                                 top_k=100, temperature=temperature, 
#                                  sample=(m < burnin)
                                )
    predicted_tokens.append(idxs[0])

In [ ]:
# tokenize text
text_ids = tokenizer_return_id(text)
num_tokens=len(text_ids)
if num_tokens >= max_len:
    text_ids_max_len = text_ids[:max_len] # truncate       
else:
    text_ids_max_len = text_ids + (max_len-num_tokens) * [pad_id] # pad       
assert len(text_ids_max_len) == max_len, len(text_ids_max_len)
batch.append(text_ids_max_len)

## Old stuff

In [ ]:
def load_file(filename):
    """
    :param filename:
    :return:
    """
    data = []
    with open(filename, "r") as f:
        for line in f.readlines():
            data.append(json.loads(line))
    return data

In [ ]:
def tokenizer_return_id(text):
    output = tokenizer(text)
    token_ids = [i for i in output['input_ids'] if i not in tokenizer.all_special_ids]
    return token_ids

In [ ]:
test_data = load_file(TEMPLAMA_NEW_DIR)
years = sorted(list(set([d["date"] for d in test_data])))
# split per year
test_data_dict = {k: {"text": [], "labels": [], "labels_ids": [], "relation": []} for k in years}

In [ ]:
d = test_data[0]
year = d["date"]
relation = d['relation']
if type(d['answer']) is not list: d['answer'] = [d['answer']]
labels_string_list = [label['name'] for label in d['answer']]
labels_ids_list = [tokenizer_return_id(string) for string in labels_string_list]
num_masks_list = [len(tokens) for tokens in labels_ids_list]

In [ ]:
all_answers = [a["name"] for a in d["answer"]]
multiple_masks = [" ".join([mask_token for _ in range(num_masks)]) for num_masks in num_masks_list]

In [ ]:
all_masked_answers = [d["query"].replace("_X_", _mask) for _mask in multiple_masks]

In [ ]:
[a["name"] for a in d["answer"]]

In [ ]:
multiple_masks = [" ".join([mask_token for _ in range(num_masks)]) for num_masks in num_masks_list]

In [ ]:
multiple_masks

In [ ]:
def load_dynamic_templama(filename):
    """
    Our dynamically-created TempLAMA version
    :return:
    """
    single_mask = False
    lowercase = False
    
    test_data = load_file(filename)
    years = sorted(list(set([d["date"] for d in test_data])))
    # split per year
    test_data_dict = {k: {"text": [], "labels": [], "labels_ids": [], "relation": []} for k in years}
    for d in tqdm(test_data):
        year = d["date"]
        relation = d['relation']
        if type(d['answer']) is not list: d['answer'] = [d['answer']]
        labels_string_list = [label['name'] for label in d['answer']]  # list of strings with all the labels
        if lowercase:
            labels_string_list = [label['name'].lower() for label in d['answer']]
        labels_ids_list = [tokenizer_return_id(string) for string in labels_string_list]  # list of lists with ids
        num_masks_list = [len(tokens) for tokens in labels_ids_list]
        """
        If we want to consider more correct labels we have to change accepted labels
        """
        # If we evaluate only single-token labels
        if single_mask:
            # check if there is answer with one mask
            if 1 in num_masks_list:
                labels_ids_with_one_mask_index = [i for i,l in enumerate(num_masks_list) if l==1]  # list of ids where answer requires one mask/token
                # accepted_labels = list(np.array(labels_string_list)[labels_ids_with_one_mask_index])
                accepted_labels_ids = np.array(labels_ids_list)[labels_ids_with_one_mask_index].tolist()
                accepted_labels = [[tokenizer.decode(label_id)] for label_id in accepted_labels_ids]
                assert len(accepted_labels)==len(accepted_labels_ids)
                assert len(accepted_labels[0])==len(accepted_labels_ids[0])
                text = d["query"].replace("_X_", mask_token)
                if lowercase:
                    text = d["query"].lower().replace("_x_", mask_token)
                
                test_data_dict[year]["text"].append(text)
                test_data_dict[year]["labels"].append(accepted_labels)
                test_data_dict[year]["labels_ids"].append(accepted_labels_ids)
                test_data_dict[year]["relation"].append(relation)
            else:
                # skip this example
                continue
        else: 
            # accept all labels
#             all_answers = [a["name"] for a in d["answer"]]
            multiple_masks = [" ".join([mask_token for _ in range(num_masks)]) for num_masks in num_masks_list]
            all_masked_queries = [d["query"].replace("_X_", _mask) for _mask in multiple_masks]
            if lowercase:
                all_masked_queries = [d["query"].lower().replace("_x_", _mask) for _mask in multiple_masks]
#                 # todo
#                 all_answers = [seq.lower() for seq in d["answer"]]
#                 # replace "_X_" with correct mask token
#                 if single_mask:
#                     text = d["query"].lower().replace("_x_", mask_token)
#                 else:
#                     num_masks = len(all_answers[0].split())
#                     multiple_masks = " ".join([mask_token for _ in range(num_masks)])
#                     text = d["query"].lower().replace("_x_", multiple_masks)
            else:
                multiple_masks = [" ".join([mask_token for _ in range(num_masks)]) for num_masks in num_masks_list]
                # replace _X_ with as many mask tokens as the correct label
                all_masked_queries = [d["query"].replace("_X_", _mask) for _mask in multiple_masks]

                # Because we have multiple correct labels of possibly different number of tokens, 
                # we now have multiple multiple texts (depending on the number of masks)
                # so we append them all separately at the test set
            
            for i in range(len(all_answers)):
                test_data_dict[year]["text"].append(all_masked_queries[i])
                test_data_dict[year]["labels"].append(labels_string_list[i])
                test_data_dict[year]["labels_ids"].append(labels_ids_list[i])
                test_data_dict[year]["relation"].append(relation)

    return test_data_dict

In [ ]:
load_dynamic_templama(TEMPLAMA_NEW_DIR)

In [ ]:
test_data = load_file(filename)

### Original TempLAMA dataset (downloaded from Google Research github repo)
- 2010-2020
- 9 relations
- 1,000 subjects per relation

In [ ]:
orig_templama = load_file(TEMPLAMA_NEW_DIR)

In [ ]:
orig_templama


Number of examples per year:

In [ ]:
Counter([x['date'] for x in orig_templama])

In [ ]:
print('In total: {} examples for all years, {} unique facts.'.format(len(orig_templama), 
      len(set([x['query'] for x in orig_templama]))))

Number of facts per relation:

In [ ]:
Counter([x['relation'] for x in orig_templama])

### Dynamic TempLAMA dataset (by running their code)
- 2019-2020
- 9 relations
- 5,000 subjects per relation

In [ ]:
reproduced_templama = load_file(TEMPLAMA_NEW_DIR)

Number of examples per quarter:


In [ ]:
Counter([x['date'] for x in reproduced_templama])

In [ ]:
print('In total: {} examples for all years, {} unique facts.'.format(len(reproduced_templama), 
      len(set([x['query'] for x in reproduced_templama]))))

Number of facts per relation:


In [ ]:
Counter([x['relation'] for x in reproduced_templama])

In [ ]:
data = load_dynamic_templama(TEMPLAMA_NEW_DIR)

In [ ]:
data['2019-Q1'].keys()

In [ ]:
for quarter in data:
    quarter_dataset = data[quarter]["text"]
    num_examples = len(quarter_dataset)
    num_relations = Counter(data['2019-Q1']['relation']).items()
    print("Quarter {} - num examples {} - relations {}".format(quarter,num_examples,num_relations))
    

In [ ]:
Counter(data['2019-Q1']['relation']).items()

In [ ]:
data_2019_q1 = data['2019-Q1']
data_2019_q2 = data['2019-Q2']

In [ ]:
next(iter(data)) 


In [ ]:
list(data.keys())

In [ ]:
unchanged_t, new_t, updated_t, deleted_t = {}, {}, {}, {}
quarters = list(data.keys())
t_1 = quarters[0]

for t in quarters[1:]:
    data_t = data[t]      # D_t
    data_t_1 = data[t_1]  # D_(t-1)

    unchanged_t[t] = {key:[] for key in data_t.keys()} 
    new_t[t] = {key:[] for key in data_t.keys()} 
    updated_t[t] = {key:[] for key in data_t.keys()} 
    deleted_t[t] = {key:[] for key in data_t.keys()} 

#     for i in range(0,len(data_t_1['text'])):
#         text_t_1 = data_t_1['text'][j]
#         if text_t_1 not in data_t['text']:
#             # text_(t+1) not in D_t
#             # add to D_deleted
#             for key in ['text', 'relation']:
#                 deleted_t[t][key].append(data_t_1[key])
                
    for i in range (0, len(data_t['text'])):
        text_t = data_t['text'][i]
        labels_ids_t = data_t['labels_ids'][i][0]
        if text_t in data_t_1['text']:
            t_1_index = data_t_1['text'].index(text_t)
            labels_inds_t_1 = data_t_1['labels_ids'][t_1_index][0]
            if labels_ids_t == labels_inds_t_1:
                # text_t = text_t-1 & label_t = label_t-1 
                # add to D_unchanged
                for key in data_t.keys():
                    unchanged_t[t][key].append(data_t[key])
            else:
                # text_t = text_(t-1) & label_t != label_(t-1)
                # add to D_updated
                for key in ['text', 'relation']:
                    updated_t[t][key].append(data_t[key])
        else:
            # text_t not in D_(t-1) texts 
            # add to D_new
            for key in ['text', 'relation']:
                new_t[t][key].append(data_t[key])
    for j in range(0,len(data_t_1['text'])):
        text_t_1 = data_t_1['text'][j]
        if text_t_1 not in data_t['text']:
            # text_(t+1) not in D_t
            # add to D_deleted
            for key in ['text', 'relation']:
                deleted_t[t][key].append(data_t_1[key])
    t_1 = t
    
    assert len(data_t['text']) == len(unchanged_t[t]['text']) + len(updated_t[t]['text']) + len(new_t[t]['text'])
    print('t={}: From total {} samples in D_t, {} are unchanged, {} are updated, {} are deleted and {} are new, compared to D_(t-1).'.format(t,
                                                                                                                             len(data_t['text']),
                                                                                                                    len(unchanged_t[t]['text']),
                                                                                                                   len(updated_t[t]['text']),
                                                                                                                                             len(deleted_t[t]['text']),
                                                                                                                   len(new_t[t]['text'])))

            
            

In [ ]:
len(data[quarters[0]]['relation'])

In [ ]:
len(updated_t['text'])

In [ ]:
len(new_t['text'])

In [ ]:
len(data_2019_q2['text'])

In [ ]:
i=0
text_data_2019_q2['text'][i]

In [ ]:
text_t = data_2019_q2['text'][i]

In [ ]:
labels_ids_t = data_2019_q2['labels_ids'][i][0]

In [ ]:
if text_t in data_2019_q1['text']:
    data_2019_q1['text']

In [ ]:
t_1_index = data_2019_q1['text'].index(text_t)

In [ ]:
labels_t_1 = data_2019_q1['labels_ids'][t_1_index][0]

In [ ]:
labels_t_1 == labels_ids_t

In [ ]:
labels_ids_t

In [ ]:
data_2019_q2.keys()

In [ ]:
unchanged_t = {key:[] for key in data_2019_q2.keys()} 

In [ ]:
unchanged_t